In [1]:
!pip install openai -q



  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [83 lines of output]
      /Users/jiezhou/miniforge3/envs/llmops-course/lib/python3.11/site-packages/setuptools/__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [17 lines of output]

In [2]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()



sk-proj-opIuvPWUabMyXZH93N3nT3BlbkFJFokcfiDljdo7ale13jpU


In [4]:
from openai import OpenAI
openai_client = OpenAI()


from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))




In [11]:
### Activity #1:

list_of_prompts = [
    system_prompt("You are judge of whether the movie review supplied by the user is positive or negative in sentiment. The output must be in JSON format only."),
    user_prompt("""
        I hated the hulk!
    """),
    assistant_prompt("""{"sentiment" : "negative"}"""),
    user_prompt("""
        I love marvel movies! 
    """),
    assistant_prompt("""{"sentiment" : "positive"}"""),
    user_prompt("""
        At times “Atlas” feels like pure pastiche, and it looks, in a fashion we’re getting used to seeing on the streamers, kind of cheap, dark, plasticky and fake, particularly in the big action sequences. Science fiction often earns its place in memory by envisioning something new and startling — but with “Atlas,” we’ve seen it all before.
    """),
    assistant_prompt("""{"sentiment" : "negative"}"""),
    user_prompt("""
        Lopez cries and rages, panics and cajoles and pretty much does everything she can to win an Oscar for best performance in a bad movie. (Too bad that category does not exist, because if it did, she’d deserve it.) It’s rare to see someone working so hard at something so hopeless. Imagine two hours of Lopez trying to deadlift 1,000 pounds, and you’d have an idea of what “Atlas” is like.
    """),
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

{"sentiment" : "negative"}

In [42]:
### Activity #2:



# system_template = """\
# You are a helpful assistant specializing in tricky mental and logical problems.
# """

# user_template = """
# You MUST do the following!
# {input} 
# Provide a clear response that is faithful to the original query.
# Make sure your answer has been verified for correctness.
# Do not assume any facts without careful analysis.
# Think through things step by step before giving the final answer.
# You will be penalized if you fail to do all of the above.
# I'm going to tip $100 for a better solution!
# """



system_template = """\
You are a helpful assistant.
"""

user_template = """
{input}
"""

HARD_PROBLEMS = [
    """Evaluate the financial costs of climate change impacting southern Spain over the next 20 years""",
    """Billy wants to get home from San Fran. before 7PM EDT. It's currently 1PM local time. Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs). Does it matter which travel option Billy selects?""",
    """Three boys agree to divide a bag of marbles in the following manner. The first boy takes one more than half the marbles. The second takes a third of the number remaining. The third boy finds that he is left with twice as many marbles as the second boy.""",
    """# There are 2 people living next to each other. Both have a pet. One of them has a dog and the other a cat. The Englishman lives in the red house and the Frenchman lives in the blue house. The owner of the blue house has a cat. Who has the dog?""",
    """A car travelling at a speed of 20 kph left a certain place at 3:00 p.m. At 5:00 p.m. another car departed from the same place at 40 kph and travelled the same route. In how many hours will the second car overtake the first?""",
]

query = HARD_PROBLEMS[1]


MODELS = ["gpt-3.5-turbo", "gpt-4o"]


list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts, model=MODELS[1])

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

pretty_print(evaluator_response)

Let's analyze both travel options to determine if Billy can make it home before 7 PM EDT.

1. **Time Zone Difference:**
   - San Francisco is in the Pacific Time Zone (PDT), which is 3 hours behind Eastern Time Zone (EDT).
   - Therefore, 1 PM PDT is equivalent to 4 PM EDT.

2. **Option 1: Fly and then take a bus:**
   - Flight time: 3 hours
   - Bus time: 2 hours
   - Total travel time: 3 + 2 = 5 hours

   If Billy starts at 1 PM PDT (4 PM EDT), adding 5 hours of travel will get him home by 9 PM EDT.

3. **Option 2: Teleport and then take a bus:**
   - Teleport time: 0 hours
   - Bus time: 1 hour
   - Total travel time: 0 + 1 = 1 hour

   If Billy starts at 1 PM PDT (4 PM EDT), adding 1 hour of travel will get him home by 5 PM EDT.

In conclusion, it does matter which travel option Billy selects. The first option (flight and bus) will not get him home before 7 PM EDT, while the second option (teleporter and bus) will get him home by 5 PM EDT, which is well before 7 PM EDT. Therefore, Billy should choose the teleporter and bus option to ensure he gets home on time.

{"clarity" : 8, "faithfulness" : 9, "correctness" : 10}

